These are scripts utilizing boto3 to view spot terminations and logs (which are currently way too long to inspect).

They are the work of [Andrew Johnston](https://github.com/asjohnston-asf).

In [1]:
import os
import boto3

In [2]:
os.environ['AWS_PROFILE'] = 'access'
batch = boto3.client('batch')

In [4]:

from os import environ
import json

import boto3
import requests

session = requests.Session()


def get_log_message(job_id):
    log_url = f'https://{domain}/{job_id}/{job_id}.log'
    # THIS IS THE BRILLIANT PART!
    # Request to only read the last line 
    headers = {'Range': 'bytes=-1000'}
    response = session.get(log_url, headers=headers)
    try:
        response.raise_for_status()
        message = response.text.split('\n')[-1]
        return message
    except Exception:
        return ''

## TIBET
#environ['AWS_PROFILE'] = 'tibet'
#state_machine_arn = 'arn:aws:states:us-west-2:015983320819:stateMachine:StepFunction-uZ45938oodIw'
#domain = 'hyp3-tibet-jpl-contentbucket-81rn23hp7ppf.s3.us-west-2.amazonaws.com'

## ACCESS
environ['AWS_PROFILE'] = 'access'
state_machine_arn = 'arn:aws:states:us-west-2:176245438256:stateMachine:StepFunction-Yg91SmVo2BvD'
domain = 'hyp3-a19-jpl-contentbucket-1wfnatpznlg8b'

sfn = boto3.client('stepfunctions')
max_results = 10


response = sfn.list_executions(stateMachineArn=state_machine_arn, statusFilter='FAILED', maxResults=max_results)
executions = response['executions']

for execution in executions:
    response = sfn.describe_execution(executionArn=execution['executionArn'])
    payload = json.loads(response['input'])
    message = get_log_message(payload['job_id'])
    print(f'{payload["job_id"]}  {response["stopDate"].strftime("%Y-%m-%d %H:%M:%S%z")}  '
          f'{payload["job_type"]:11}  {payload["user_id"]:25}  {message}')

ConnectionError: HTTPSConnectionPool(host='hyp3-a19-jpl-contentbucket-1wfnatpznlg8b', port=443): Max retries exceeded with url: /6e7be333-f794-4709-bc45-feec6f3362cb/6e7be333-f794-4709-bc45-feec6f3362cb.log (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x111e5cf50>: Failed to resolve 'hyp3-a19-jpl-contentbucket-1wfnatpznlg8b' ([Errno 8] nodename nor servname provided, or not known)"))

In [3]:
# tibet
#queue = 'BatchJobQueue-8YxF2wT5tW3nPcry'
# access
queue = 'BatchJobQueue-fzRFEEEvL4l1jLRp'

In [4]:
params = {
    'jobQueue': queue,
    'jobStatus': 'FAILED',
    'maxResults': 100,
}

terminations = []
exited = []

paginator = batch.get_paginator('list_jobs')
page_iterator = paginator.paginate(**params)
for k, page in enumerate(page_iterator):
    print(k)
    job_ids = [job['jobId'] for job in page['jobSummaryList']]
    response = batch.describe_jobs(jobs=job_ids)
    for job in response['jobs']:
        print(f'{job["jobName"]} {job["statusReason"]}')
    if k > 10:
        break

0
78bcd734-5cad-4781-bf9f-43c719f3260b Host EC2 (instance i-08376ac0f4eb8f698) terminated.
d79958b9-747a-4d84-87bc-3be68d276183 Host EC2 (instance i-0dbd7b0270c3c36dd) terminated.
55bf0f19-aa2a-497a-87c8-cf3ad2a5fe6f Host EC2 (instance i-09e9e59cdef82be70) terminated.
26def07a-d69a-4da3-b873-0e8f789878b6 Host EC2 (instance i-00d69ba22eedf31aa) terminated.
d1ea4a00-1f9f-4fdc-aa1d-4f3449a8a4d3 Host EC2 (instance i-07265bee998fde4a9) terminated.
1edd7f18-a660-4f61-9884-8fdb8934a38c Host EC2 (instance i-0a001169db75d0e67) terminated.
76cf93bf-000d-4a9d-a429-f9e5d6904c3f Host EC2 (instance i-0baaa5a4f637016fa) terminated.
5c0e2bdf-b737-42e4-9529-7c04a478c8e7 Host EC2 (instance i-0234b1322fe79f505) terminated.
813e67de-8179-475a-9170-ca1c08e23934 Host EC2 (instance i-00cacb9786d8e71a3) terminated.
ca7712b8-6dd7-4c24-bb3f-eee4fdb27ac6 Host EC2 (instance i-01916c4e7140198e7) terminated.
5c95e170-507c-47f2-ad25-08e8cab690a7 Essential container in task exited
82d73495-8eb5-4680-a98c-e6523a2923bc

ClientError: An error occurred (TooManyRequestsException) when calling the DescribeJobs operation (reached max retries: 4): Too Many Requests

In [11]:
params = {
    'jobQueue': queue,
    'jobStatus': 'FAILED',
    'maxResults': 100,
}

terminations = []
exited = []

paginator = batch.get_paginator('list_jobs')
page_iterator = paginator.paginate(**params)
for k, page in enumerate(page_iterator):
    print(k)
    job_ids = [job['jobId'] for job in page['jobSummaryList']]
    response = batch.describe_jobs(jobs=job_ids)
    terminations += [job['jobName'] for job in response['jobs'] if 'terminated.' in job["statusReason"]]
    exited += [job['jobName'] for job in response['jobs'] if 'task exited' in job["statusReason"]]
    if k > 10:
        break

0


ClientException: An error occurred (ClientException) when calling the DescribeJobs operation: Error executing request, Exception : At least one JobId is required., RequestId: ce3290af-d0be-4bef-b762-c62543eb285f

In [9]:
float(len(terminations)) / (len(exited) + len(terminations))

0.988950276243094

In [11]:
import hyp3_sdk
import datetime

now = datetime.datetime.now()
submit_time_approx = now - datetime.timedelta(hours=96)


hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu')
jobs = hyp3_isce.find_jobs(start=submit_time_approx, user_id='access_cloud_based_insar')
print(jobs)

97689 HyP3 Jobs: 0 succeeded, 5496 failed, 1245 running, 90948 pending.


In [13]:
float(len(terminations)) / (len(exited) + len(terminations))

0.9947916666666666

In [14]:
(len(exited) + len(terminations))

192

ClientError: An error occurred (AccessDeniedException) when calling the ListExecutions operation: User: arn:aws:iam::015983320819:user/SRV-hyp3-tibet-jpl is not authorized to access this resource